# Exerimental Design with Retail Data

## Table of Contents:
* [About the Dataset](#introduction)
* [Proposal](#proposal)
    - [General Product and Category Breakdown](#gen)
    - [Pricing Explorations](#price)
* [Question 1: Shoud a buyer focus more on men's or women's fashion to maximize sales price?](#q1)
* [Question 2. Are discounts on products affected by season?](#q2)
* [Question 3. How has the price of apparel items carried by the company changed over time? Have prices for some styles changed more than others?](#q3)
* [Dataset Challenges and Future Work](#dc)

 ## About the Dataset <a name="introduction"></a>
 <br>
 
This sample data set was originally uploaded by Swayanjeet Mishra as a Kaggle test dataset. The focus of the dataset is a company that sells grocery products wholesale to other suppliers. I chose this dataset for the experimental design capstone because it contains datapoints for just over 42,000 customers with information on whether they made a purchase after being contacted by a salesperson vs. contacted through a marketing email.

All sales data is for the U.S., and custoemrs are broken down by state, so we can evaluate the effects of the two marketing approaches by region as well as more broadly. For the sake of this experimental design, I refer to the company as BevCo because they mostly sell beverages.

## Proposal <a name='proposal'></a>
<br>

In the past, BevCo has maintained a large salesforce and used sales calls as the first contact for potential customers. This practice has been effective but expensive and hard to scale. In recent years, BevCo has tested email marketing and had some success. Current marketing efforts are divide rougly evenly, with half of initial outreach made through email (n 20584) and half through direct calls (n = 21901).

BevCo is anxious to cut cost by reducing sales calls if they can prove that email marketing is at least as effective as their older, more labor intensive apprach. They have asked Marketing to create a new email campaign to test whether customer conversion rate can be maintained without the influence of direct sales calls.

In [147]:
import pandas as pd
import requests
import io


url = 'https://raw.githubusercontent.com/colettegabriel/capstone_2/master/Powersell%20DATA.csv?token=AFQPQ3ZKRKUMNDPVOXRCB5K46WKY4'

data_file = requests.get(url).content
df = pd.read_csv(io.StringIO(data_file.decode('ISO-8859-1')))
pd.options.mode.chained_assignment = None
df.drop(['Brand', 
         'Customer Size By Number of Employees',
         'Competitor Type',
         'Anonymous Variable 1',
         'Anonymous Variable 2', 
         'Anonymous Variable 3',
         'Anonymous Variable 4'], axis=1, inplace=True)

In [206]:
def cat_filter(df, category, filter):
    cat_filter = df.loc[df[category]== filter]
    return cat_filter

def state_count(df, filter):
    yes_count = df[(df['First Contact Type'] == 'Direct Contact by Salesperson') &
               (df['State'] == filter) & 
               (df['Sale'] == 'Yes')].count()
    yes_count = yes_count['Sale']
    no_count = df[(df['First Contact Type'] == 'Direct Contact by Salesperson') &
               (df['State'] == filter) & 
               (df['Sale'] == 'No')].count()
    no_count = no_count['Sale']
#    yes_count = df[(df[category1] == filter1) & (df['State'] == 'Yes')]['Type']
 #   no_count = df[(df[category1] == filter1) & (df['State'] == 'No')]['Type']
  #  print('State = {}\yes = {}\nno = {}'.format(filter, yes_count, no_count))
    return yes_count, no_count

In [149]:
call_count = len(df[df['First Contact Type'] == 'Direct Contact by Salesperson'])
email_count = len(df[df['First Contact Type'] == 'Email'])

calls = cat_filter(df, 'First Contact Type', 'Direct Contact by Salesperson')
call_sale = len(calls[calls['Sale'] == 'Yes'])
call_percent = (call_sale/call_count)*100

emails = cat_filter(df, 'First Contact Type', 'Email')
email_sale = len(emails[emails['Sale'] == 'Yes'])
email_percent = (email_sale/email_count)*100

print('The current success rate for direct contact with a sales person is {} percent.'.format(round(salesper_percent, 2)))
print('Current success rate for email marketing is {} percent.'.format(round(email_percent, 2)))



The current success rate for direct contact with a sales person is 27.57 percent.
Current success rate for email marketing is 18.39 percent.


In [240]:
calls_region = calls.groupby(['State']).count().reset_index()
emails_region = emails.groupby(['State']).count().reset_index()

In [250]:
states = ['Arizona', 'California', 'Oregon', 'Washington', 'Texas', 'Utah', 'Montana']
state_yes = dict()
state_no = dict()
calls_state = dict()
state_sales_per = dict()

for state in states:
    #calls_state[state] = calls_region[calls_region['State'] == state]['Type']
    state_yes[state], state_no[state] = state_count(df, state)
    state_sales_per[state] = state_yes[state]/(state_yes[state]+state_no[state])
    

In [251]:
state_sales_per

{'Arizona': 0.237807335751713,
 'California': 0.2764764104256021,
 'Oregon': 0.2718162839248434,
 'Washington': 0.24484439445069367,
 'Texas': 0.21660649819494585,
 'Utah': 0.3382200758882373,
 'Montana': 0.25200400801603207}

In [242]:
calls_state

{'Arizona': 0    2481
 Name: Type, dtype: int64, 'California': 1    3031
 Name: Type, dtype: int64, 'Oregon': 3    2395
 Name: Type, dtype: int64, 'Washington': 6    2667
 Name: Type, dtype: int64, 'Texas': 4    2216
 Name: Type, dtype: int64, 'Utah': 5    5798
 Name: Type, dtype: int64, 'Montana': 2    1996
 Name: Type, dtype: int64}

In [252]:
reg_sales_calls

,State,Sale,Opportunity Number,Type,Country,First Contact Type,Date of First Contact,Date of Giving Proposal,Date of Closing,Number of Contacts,Opportunity Amount (in USD),Customer Size By Revenue,Revenue From Customer (in USD)
0,Arizona,No,1891,1891,1891,1891,1891,1891,1891,1891,1891,1891,1891
1,Arizona,Yes,590,590,590,590,590,590,590,590,590,590,590
2,California,No,2193,2193,2193,2193,2193,2193,2193,2193,2193,2193,2193
3,California,Yes,838,838,838,838,838,838,838,838,838,838,838
4,Montana,No,1493,1493,1492,1493,1493,1493,1493,1493,1493,1493,1493
5,Montana,Yes,503,503,503,503,503,503,503,503,503,503,503
6,Oregon,No,1744,1744,1742,1744,1744,1744,1744,1744,1744,1744,1744
7,Oregon,Yes,651,651,651,651,651,651,651,651,651,651,651
8,Texas,No,1736,1736,1736,1736,1736,1736,1736,1736,1736,1736,1736
9,Texas,Yes,480,480,480,480,480,480,480,480,480,480,480
